## Exploring the Relationship of Income, States, and Obesity using Web Scraping

This notebook contains demographic data with income and states, and the demographic data with obesity and states from Wikipedia:
https://en.wikipedia.org/wiki/Obesity_in_the_United_States

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

%matplotlib inline
plt.rcParams['font.size'] = 14
pd.set_option('max_columns', 30) # to be able to see all columns

## Read in data without unknown income

In [2]:
pd.set_option('max_rows', 20)
pd.set_option('max_columns', 30)
drop_unknown_income = pd.read_csv('dropped_income.csv')

## Get only columns state and income

In [5]:
income_state = drop_unknown_income.copy()
income_state = income_state[['state', 'income']]
income_state.head()

,state,income
0,IA,E
1,IA,E
2,NE,F
3,IA,F
4,NE,I


## Convert income from categorical to numeric data

In [6]:
def f(row):
    if row['income'] == 'A':
        val = 1
    elif row['income'] == 'B':
        val = 2
    elif row['income'] == 'C':
        val = 3
    elif row['income'] == 'D':
        val = 4
    elif row['income'] == 'E':
        val = 5
    elif row['income'] == 'F':
        val = 6
    elif row['income'] == 'G':
        val = 7
    elif row['income'] == 'H':
        val = 8
    elif row['income'] == 'I':
        val = 9
    elif row['income'] == 'J':
        val = 10
    elif row['income'] == 'K':
        val = 11
    else:
        val = 12
    return val

In [7]:
income_state['income_number'] = income_state.apply(f, axis=1)
# df['C'] = df.apply(f, axis=1)
income_state.head()


,state,income,income_number
0,IA,E,5
1,IA,E,5
2,NE,F,6
3,IA,F,6
4,NE,I,9


## Calculate the average income group by state

In [8]:
income_order = income_state.groupby(['state']).mean().sort('income_number')
income_order.head()
# len(income_order.index)

/Users/jenniferchen/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,income_number
state,
WV,3.971060
ME,4.345455
TN,4.359804
MS,4.426220
NM,4.430636


## Make a column with the ordinal income by state

In [9]:
order_number = pd.DataFrame(range(1,52), columns=['order'], index=income_order.index)
order_number.head()

,order
state,
WV,1
ME,2
TN,3
MS,4
NM,5


In [10]:
result = pd.concat([income_order, order_number], axis=1, ignore_index=True)
result.head()

,0,1
state,,
WV,3.971060,1
ME,4.345455,2
TN,4.359804,3
MS,4.426220,4
NM,4.430636,5


In [11]:
result['state'] = result.index
result.columns = ['avg', 'avg_order', 'state']
del result['avg']
result.head()

,avg_order,state
state,,
WV,1,WV
ME,2,ME
TN,3,TN
MS,4,MS
NM,5,NM


## Merge the ordinal average income by state back to the original data frame

In [12]:
pd.set_option('max_rows', 20)
merge_result = pd.merge(income_state, result, on='state')
merge_result.head()

,state,income,income_number,avg_order
0,IA,E,5,12
1,IA,E,5,12
2,IA,F,6,12
3,IA,D,4,12
4,IA,C,3,12


## Calculate the correlation between state and ordinal average income by state

In [56]:
merge_result.income_number.corr(merge_result.avg_order, method='spearman')

0.1487984160700315

In [58]:
merge_result.income_number.corr(merge_result.avg_order, method='pearson')

0.15159863035393076

# Correlation between Obesity and Income

In [1]:
from bs4 import BeautifulSoup
import urllib3
import requests

### Extract Table of Obesity by State from Wikipedia

In [3]:
wiki = 'https://en.wikipedia.org/wiki/Obesity_in_the_United_States'
header = {'User-Agent': 'Mozilla/5.0'}
r = requests.get(wiki, headers=header)
soup = BeautifulSoup(r.text, "lxml")
table = soup.find("table", { "class" : "wikitable sortable" })
obesity_state = {'state': [], 'obesity_rank': []}

for row in table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 5:
        state = cells[0].find(text=True)
        obesity_rank = cells[4].find(text=True)
        obesity_state['state'].append(state)
        obesity_state['obesity_rank'].append(obesity_rank)
        
print(obesity_state)

{'state': [u'Alabama', u'Alaska', u'Arizona', u'Arkansas', u'California', u'Colorado', u'Connecticut', u'District of Columbia', u'Delaware', u'Florida', u'Georgia', u'Hawaii', u'Idaho', u'Illinois', u'Indiana', u'Iowa', u'Kansas', u'Kentucky', u'Louisiana', u'Maine', u'Maryland', u'Massachusetts', u'Michigan', u'Minnesota', u'Mississippi', u'Missouri', u'Montana', u'Nebraska', u'Nevada', u'New Hampshire', u'New Jersey', u'New Mexico', u'New York', u'North Carolina', u'North Dakota', u'Ohio', u'Oklahoma', u'Oregon', u'Pennsylvania', u'Rhode Island', u'South Carolina', u'South Dakota', u'Tennessee', u'Texas', u'Utah', u'Vermont', u'Virginia', u'Washington', u'West Virginia', u'Wisconsin', u'Wyoming'], 'obesity_rank': [u'3', u'14', u'40', u'9', u'41', u'51', u'49', u'43', u'22', u'39', u'12', u'50', u'31', u'26', u'11', u'19', u'23', u'7', u'4', u'34', u'28', u'48', u'10', u'30', u'1', u'13', u'45', u'18', u'36', u'35', u'42', u'38', u'37', u'16', u'21', u'17', u'8', u'29', u'24', u'46', 

In [6]:
# Convert table from dictionary to data frame
obesity_state_df = pd.DataFrame(obesity_state)
obesity_state_df.columns = ['obesity_rank', 'state_full']
obesity_state_df.head()

,obesity_rank,state_full
0,3,Alabama
1,14,Alaska
2,40,Arizona
3,9,Arkansas
4,41,California


### Extract List of U.S. state abbreviations from Wikipedia

In [179]:
wiki = 'https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations'
header = {'User-Agent': 'Mozilla/5.0'}
r = requests.get(wiki, headers=header)
soup = BeautifulSoup(r.text)
table = soup.find("table", { "class" : "wikitable sortable" })
abbr_state = {'state': [], 'state_full': []}

for row in table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 10:
        if len(cells[3]) == 1:
            state = cells[3].find(text=True)
            state_full = cells[0].find("a").text#, text=True)#.find(text=True)

            abbr_state['state'].append(state)
            abbr_state['state_full'].append(state_full)
        
print(abbr_state)

{'state': [u'US', u'AL', u'AK', u'AZ', u'AR', u'CA', u'CO', u'CT', u'DE', u'DC', u'FL', u'GA', u'HI', u'ID', u'IL', u'IN', u'IA', u'KS', u'KY', u'LA', u'ME', u'MD', u'MA', u'MI', u'MN', u'MS', u'MO', u'MT', u'NE', u'NV', u'NH', u'NJ', u'NM', u'NY', u'NC', u'ND', u'OH', u'OK', u'OR', u'PA', u'RI', u'SC', u'SD', u'TN', u'TX', u'UT', u'VT', u'VA', u'WA', u'WV', u'WI', u'WY', u'AS', u'GU', u'MP', u'PR', u'VI', u'UM', u'FM', u'MH', u'PW'], 'state_full': [u'United States of America', u'Alabama', u'Alaska', u'Arizona', u'Arkansas', u'California', u'Colorado', u'Connecticut', u'Delaware', u'District of Columbia', u'Florida', u'Georgia', u'Hawaii', u'Idaho', u'Illinois', u'Indiana', u'Iowa', u'Kansas', u'Kentucky', u'Louisiana', u'Maine', u'Maryland', u'Massachusetts', u'Michigan', u'Minnesota', u'Mississippi', u'Missouri', u'Montana', u'Nebraska', u'Nevada', u'New Hampshire', u'New Jersey', u'New Mexico', u'New York', u'North Carolina', u'North Dakota', u'Ohio', u'Oklahoma', u'Oregon', u'Penns

In [154]:
# Convert table from dictionary to data frame
state_abbr = pd.DataFrame(abbr_state)
state_abbr

,state,state_full
0,US,United States of America
1,AL,Alabama
2,AK,Alaska
3,AZ,Arizona
4,AR,Arkansas
5,CA,California
6,CO,Colorado
7,CT,Connecticut
8,DE,Delaware
9,DC,District of Columbia


### Merge Table of Obesity Rank and List of U.S. State Abbreviations

In [159]:
pd.set_option('max_rows', 20)
merge_obesity_state = pd.merge(obesity_state_df, state_abbr, on='state_full')
merge_obesity_state

,obesity_rank,state_full,state
0,3,Alabama,AL
1,14,Alaska,AK
2,40,Arizona,AZ
3,9,Arkansas,AR
4,41,California,CA
5,51,Colorado,CO
6,49,Connecticut,CT
7,43,District of Columbia,DC
8,22,Delaware,DE
9,39,Florida,FL


In [166]:
# Delete the full name of states
obesity_rank_df = merge_obesity_state.copy()
del obesity_rank_df['state_full']
obesity_rank_df

,obesity_rank,state
0,3,AL
1,14,AK
2,40,AZ
3,9,AR
4,41,CA
5,51,CO
6,49,CT
7,43,DC
8,22,DE
9,39,FL


### Merge the Obesity Rank with Income by State

In [180]:
pd.set_option('max_rows', 20)
merge_state_obesity = pd.merge(income_state, obesity_rank_df, on='state')
merge_state_obesity.obesity_rank = pd.to_numeric(merge_state_obesity.obesity_rank)
merge_state_obesity

,state,income,income_number,obesity_rank
0,IA,E,5,19
1,IA,E,5,19
2,IA,F,6,19
3,IA,D,4,19
4,IA,C,3,19
5,IA,D,4,19
6,IA,F,6,19
7,IA,B,2,19
8,IA,H,8,19
9,IA,I,9,19


### Calculate the Correlations between Income and Obesity by State

In [173]:
merge_state_obesity.income_number.corr(merge_state_obesity.obesity_rank, method='pearson')

0.074523345152696721

In [175]:
merge_state_obesity.income_number.corr(merge_state_obesity.obesity_rank, method='spearman')

0.07430142629140056